In [1]:
%load_ext lab_black

import pandas as pd

In [2]:
traffic = pd.read_parquet("traffic.parquet")
print(sum(traffic.flights), "flights")
traffic

194735188 flights


,typecode_oag,typecode_icao,origin,destination,km,flights
0,737,B38M,RKSS,RKPC,451,23983
6,737,B736,RKSS,RKPC,451,23983
9,737,B739,RKSS,RKPC,451,23983
8,737,B738,RKSS,RKPC,451,23983
7,737,B737,RKSS,RKPC,451,23983
...,...,...,...,...,...,...
543066,737,B737,HAAB,HHSB,620,1
543065,737,B736,HAAB,HHSB,620,1
543064,737,B735,HAAB,HHSB,620,1
543063,737,B734,HAAB,HHSB,620,1


In [3]:
# Filter out flights with invalid GC distances (execeding max flight distance of a specific aircraft type)
import numpy as np
from scipy import stats

traffic = pd.read_parquet("traffic.parquet")
traffic["z_score"] = np.NAN
typecode = "typecode_icao"

for ac in traffic[typecode].unique():
    traffic.loc[traffic[typecode] == ac, "z_score"] = abs(
        stats.zscore(traffic.loc[traffic[typecode] == ac, "km"])
    )
filtered = traffic.loc[(traffic.z_score < 5) | (traffic.z_score.isna())]
removed = traffic.loc[traffic.z_score > 5]
print(
    "{:,.0f} ({:.2f}% of total) routes removed".format(
        len(removed), len(removed) / len(traffic) * 100
    )
)

# for ac in filtered[typecode].unique():
#     filtered.loc[filtered[typecode] == ac, "z_score"] = abs(
#         stats.zscore(filtered.loc[filtered[typecode] == ac, "km"])
#     )
# filtered = filtered.loc[(filtered.z_score < 6) | (filtered.z_score.isna())]
# removed = filtered.loc[traffic.z_score > 6]
# print(
#     "{:,.0f} ({:.2f}% of total) routes removed".format(
#         len(removed), len(removed) / len(traffic) * 100
#     )
# )
traffic = filtered.reset_index(drop=True).drop("z_score", axis=1)
# # traffic.to_parquet("traffic.parquet")
traffic

500 (0.09% of total) routes removed


,typecode_oag,typecode_icao,origin,destination,km,flights
0,737,B38M,RKSS,RKPC,451,23983
1,737,B736,RKSS,RKPC,451,23983
2,737,B739,RKSS,RKPC,451,23983
3,737,B738,RKSS,RKPC,451,23983
4,737,B737,RKSS,RKPC,451,23983
...,...,...,...,...,...,...
564475,737,B737,HAAB,HHSB,620,1
564476,737,B736,HAAB,HHSB,620,1
564477,737,B735,HAAB,HHSB,620,1
564478,737,B734,HAAB,HHSB,620,1


In [29]:
km = (
    traffic.groupby(["origin", "destination", "typecode_oag"])
    .first()
    .reset_index()
    .rename(columns={"km": "km_gc"})
)
km

,origin,destination,typecode_oag,typecode_icao,km_gc,flights
0,03N,PKMJ,D28,D228,488,53
1,0AK,PABE,CNA,C172,140,666
2,0AK,PAMO,CNA,C172,43,666
3,16A,4A2,CNA,C172,10,1387
4,16A,PABE,CNA,C172,35,390
...,...,...,...,...,...,...
118371,rjns,ZSHC,32S,A319,1735,206
118372,rjns,ZSHC,330,A332,1735,2
118373,rjns,ZSNB,32S,A318,1661,103
118374,rjns,ZSPD,32S,A321,1577,65


In [32]:
km = (
    km.groupby(["origin", "destination"])
    .agg({"km_gc": "first", "flights": "sum"})
    .reset_index()
)
km = km.assign(total_km=km.km_gc * km.flights)
km

,origin,destination,km_gc,flights,total_km
0,03N,PKMJ,488,53,25864
1,0AK,PABE,140,666,93240
2,0AK,PAMO,43,666,28638
3,16A,4A2,10,1387,13870
4,16A,PABE,35,402,14070
...,...,...,...,...,...
66358,rjns,ROAH,1387,365,506255
66359,rjns,YNT,1571,6,9426
66360,rjns,ZSHC,1735,208,360880
66361,rjns,ZSNB,1661,103,171083


In [35]:
km.to_csv("oag_2018_km.csv")